In [1]:
import pandas as pd
from ggmodel_dev.models.utils import all_model_dictionary, all_model_properties_df


In [18]:
value = 'BE2_model'
all_model_properties_df.query(f'model == "{value}"').description.values[0]

'Land use change is a consequence of changes in a country’s food demand. The total food demand for 97 different food groups are calculated based on the total calorie demand, population, food waste reduction policies and other types of food demand.  From the total crop-based food demands, the total cropland demand was computed by dividing it by the total crop yields. However, some minor differences occurred as the food groups calculated did not exactly match the crop types in FAOSTAT. Thus, an additional parameter was used to correct this. With the total cropland demand, the change in land use was calculated by comparing it to the previous year. If the cropland area increased, it was assumed that this caused a decrease in the inactive or fallow land area. When cropland area decreases, the total inactive land or fallow areas will increase. A reforestation scenario set by the user determines the share of inactive lands to be converted to forests. Both the fallow land and forest area were 

In [14]:
all_model_dictionary['EW1_partial_model']

In [9]:
value = 'Landuse'


[{'label': 'BE2_model', 'value': 'Change in Land Use'},
 {'label': 'BE3_model', 'value': 'Change in above group Biomass'},
 {'label': 'SL1_model', 'value': 'Nutrient Balance'},
 {'label': 'GE3_model', 'value': 'Agricultural Non-CO2 Emissions'}]